In [ ]:
import os
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()
aval_api_key=os.getenv("AVALAI_API_KEY")


In [ ]:
client = OpenAI(    
    api_key=aval_api_key,
    base_url="https://api.avalai.ir/v1"
    )

In [ ]:
instructions = """
You are a personal math tutor. When asked a math question,
write and run code using the python tool to answer the question.
"""

response = client.responses.create(
    model="gpt-4o-mini",
    tools=[
        {
            "type": "code_interpreter",
            "container": {"type": "auto", "memory_limit": "4g"}
        }
    ],
    instructions=instructions,
    input="I need to solve the equation 3x + 11 = 14. Can you help me?",
)

print(response.output)

In [ ]:
display(Markdown(response.output_text))

In [ ]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="Generate 10 random numbers and compute their average.",
    instructions=instructions,
    tools=[
        {
            "type": "code_interpreter",
            "container": {"type": "auto", "memory_limit": "4g"}
        }
    ]
    )
response

In [ ]:
display(Markdown(response.output_text))

# Code Generation

In [ ]:
def generate_code(user_request):
    response = client.responses.create(
        model="gpt-4o-mini",
        input=f"""
            Write Python code that satisfies the following request:

            {user_request}

            The code must define a function:

                def solution(input_data):

            that returns the output of the program.
            Only output valid Python code — no explanations.
            Response only in Python code. Use comments sparingly and do not provide any explanation other than occasional comments.
        """,
    )
    
    return response.output_text

In [ ]:
code_result = generate_code("create a bubble sort")

In [ ]:
display(Markdown(code_result))

In [ ]:
def execute_code(code, user_input):
    # Ask the LLM to run this code via the tool
    response = client.responses.create(
        model="gpt-4o-mini",
        instructions="Run the provided Python code with the user input and return the result.",
        tools=[
            {
                "type": "code_interpreter",
                "container": {"type": "auto", "memory_limit": "4g"}
            }
        ], 
        input=f"Generated code: {code} and here is the user input {user_input}"    
    )

    return response.output_text

In [ ]:
execution_result = execute_code(code_result, [200, 400, -100, -4, 324])

In [ ]:
display(Markdown(execution_result))

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# 🧠 LLM Python Code Generator + Code Interpreter Runner")

    # Input prompt
    user_prompt = gr.Textbox(
        label="Describe the code you want generated",
        placeholder="Example: Implement bubble sort in Python.",
        lines=3
    )
    generate_btn = gr.Button("Generate Code")

    # LLM-generated code display
    code_box = gr.Code(language="python", label="Generated Code")

    # Input data from the user
    user_data = gr.Textbox(
        label="Input to pass to solution(input_data)",
        placeholder="[3, 1, 9, -2]"
    )
    run_btn = gr.Button("Run Code")

    output_box = gr.Textbox(
        label="Execution Result",
        lines=3
        )

    # Bind actions
    generate_btn.click(generate_code, inputs=user_prompt, outputs=code_box)
    run_btn.click(execute_code, inputs=[code_box, user_data], outputs=output_box)

demo.launch()
